In [3]:
# 문자열 토큰화 + LSTM 감성분류
from tensorflow.keras.preprocessing.text import Tokenizer

samples = ['The cat say on the mat.', 'The dog ate my homework.']    # list type

# token 처리 1
token_index = {}
for sam in samples:
  for word in sam.split(sep=' '):
    if word not in token_index:
      #print(word)
      token_index[word] = len(token_index)

print(token_index)    # {'The': 0, 'cat': 1, 'say': 2, 'on': 3, 'the': 4, 'mat.': 5, 'dog': 6, 'ate': 7, 'my': 8, 'homework.': 9}

print()
# token 처리 2 - word index
tokenizer = Tokenizer()    # Tokenizer(num_words=5), 빈도가 높은 일부(여기서는 5개) 자료만 자료에 참여
tokenizer.fit_on_texts(samples)
token_seq = tokenizer.texts_to_sequences(samples)
print(token_seq)      # [[1, 2, 3, 4, 1, 5], [1, 6, 7, 8, 9]]
print(tokenizer.word_index)         # {'the': 1, 'cat': 2, 'say': 3, 'on': 4, 'mat': 5, 'dog': 6, 'ate': 7, 'my': 8, 'homework': 9}

{'The': 0, 'cat': 1, 'say': 2, 'on': 3, 'the': 4, 'mat.': 5, 'dog': 6, 'ate': 7, 'my': 8, 'homework.': 9}

[[1, 2, 3, 4, 1, 5], [1, 6, 7, 8, 9]]
{'the': 1, 'cat': 2, 'say': 3, 'on': 4, 'mat': 5, 'dog': 6, 'ate': 7, 'my': 8, 'homework': 9}


In [4]:
# mode='count' 등장 횟수, 'tfidf', 'freq' : 빈도수, 'binary' : 있으면 1없으면 0
token_mat = tokenizer.texts_to_matrix(samples, mode='binary')
print(token_mat)
print(tokenizer.word_counts)
print(tokenizer.document_count)
print(tokenizer.word_docs)

from tensorflow.keras.utils import to_categorical
token_seq = to_categorical(token_seq[1], num_classes=6)
print(token_seq)


[[0. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 1. 1. 1.]]
OrderedDict([('the', 3), ('cat', 1), ('say', 1), ('on', 1), ('mat', 1), ('dog', 1), ('ate', 1), ('my', 1), ('homework', 1)])
2
defaultdict(<class 'int'>, {'say': 1, 'on': 1, 'the': 2, 'mat': 1, 'cat': 1, 'my': 1, 'homework': 1, 'dog': 1, 'ate': 1})


IndexError: index 6 is out of bounds for axis 1 with size 6

In [5]:
# 초간단 영화 리뷰 자료로 감성분석
import numpy as np
docs = ['너무 재미있어요', '또 보고 싶어요', '참 잘 만든 영화네요', '친구에게 추천할래요', '배우가 멋져요',
        '별로에요', '최악이에요', '연기가 어색하네요', '시간만 버렸어요', '지루하고 재미없어요']
classes = np.array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])
token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)

x = token.texts_to_sequences(docs)
print('토큰화 결과 : ', x)

from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_x = pad_sequences(x, 4)    # 패딩 : 서로 다른 길이의 데이터를 가장 긴 데이터의 길이로 맞춘다.
# 병렬 연산을 위해서 여러 문장의 길이를 임의로 동잃하게 맞춰주는 작업이 필요.
print(padded_x)


{'너무': 1, '재미있어요': 2, '또': 3, '보고': 4, '싶어요': 5, '참': 6, '잘': 7, '만든': 8, '영화네요': 9, '친구에게': 10, '추천할래요': 11, '배우가': 12, '멋져요': 13, '별로에요': 14, '최악이에요': 15, '연기가': 16, '어색하네요': 17, '시간만': 18, '버렸어요': 19, '지루하고': 20, '재미없어요': 21}
토큰화 결과 :  [[1, 2], [3, 4, 5], [6, 7, 8, 9], [10, 11], [12, 13], [14], [15], [16, 17], [18, 19], [20, 21]]
[[ 0  0  1  2]
 [ 0  3  4  5]
 [ 6  7  8  9]
 [ 0  0 10 11]
 [ 0  0 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0 20 21]]


In [ ]:
# 모델
word_size = len(token.word_index) + 1       # embedding에서는 전체 word_size + 1을 해줘야한다.
print(word_size)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, LSTM
model = Sequential()
model.add(Embedding(word_size, output_dim=8 , input_length=4))
model.add(LSTM(32, activation='tanh'))
model.add(Flatten())    # FC Layer
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_c'
                                     'rossentropy', metrics=['accuracy'])
model.fit(padded_x, classes, epochs=10, verbose=1)
print('acc : ',  model.evaluate(padded_x, classes)[1])
print('pred : ', model.predict(padded_x).flatten())
print('real : ', classes)